In [4]:
from mido import MidiFile
import os


In [5]:
filepath = os.path.join(os.getcwd(),"midis", "mozart", "mz_311_1.mid")
mid = MidiFile(filepath, clip=True)
print(mid)

<midi file '/Users/lena/git/NLP_study_buddy/midis/mozart/mz_311_1.mid' type 1, 10 tracks, 7493 messages>
